In [1]:
#!/usr/bin/python3
# -*-coding:utf-8
'''
Created on Fri Dec 1 22:22:35 2017

@author: Ray

'''
import time
import pandas as pd
import numpy as np
from tqdm import tqdm
import os
import utils # written by author
from glob import glob
from datetime import datetime, timedelta
import multiprocessing as mp
import gc # for automatic releasing memory




In [2]:
##################################################
# Load  
##################################################
col = ['msno','plan_list_price','payment_plan_days','actual_amount_paid','payment_method_id','transaction_date',]
transactions = utils.read_multiple_csv('../../input/preprocessed_data/transactions') # 20,000,000


100%|██████████| 4/4 [00:28<00:00,  7.05s/it]


In [3]:
#==============================================================================
print('reduce memory')
#==============================================================================
utils.reduce_memory(transactions)

reduce memory


100%|██████████| 9/9 [00:00<00:00, 2568.64it/s]


In [57]:
transactions.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 22964064 entries, 0 to 5741015
Data columns (total 9 columns):
msno                      object
payment_method_id         int8
payment_plan_days         int16
plan_list_price           int16
actual_amount_paid        int16
is_auto_renew             int8
transaction_date          object
membership_expire_date    object
is_cancel                 int8
dtypes: int16(3), int8(3), object(3)
memory usage: 897.9+ MB


In [4]:
"is_cancel_chance"
"is_cancel_ratio"

'is_cancel_ratio'

In [9]:
transactions[transactions.is_cancel == 1]

,msno,payment_method_id,payment_plan_days,plan_list_price,actual_amount_paid,is_auto_renew,transaction_date,membership_expire_date,is_cancel
76,++/Gw1B9K+XOlB3hLTloeUK2QlCa2m+BJ8TrzGf7djI=,40,30,149,149,1,2015-07-21,2015-08-16,1
86,++/TR7WI15q2ZCtOXmoap7jR+kEhbMVE5swOqsfqpqI=,11,30,149,149,1,2015-08-26,2015-09-25,1
135,++/gTmVgKUbNFmsTiriZdWV1uZIrLXCUiEWN0fEU6BM=,41,30,149,149,1,2016-02-15,2016-02-15,1
181,++0KpReZV9DjNUlP6aQhK3MDLWaxbA6fvZKh4UaFc3w=,41,30,149,0,1,2015-08-24,2015-09-23,1
188,++0nOC7BmrUTtcSboRORfg6ZXTajnBDt1f/SEgH6ONo=,16,30,149,149,1,2015-12-12,2015-12-12,1
239,++1SsMUpPBTruO9rzPk9p/OpgMorz3TzRd1howM1fqw=,41,30,149,0,1,2015-10-18,2015-10-20,1
241,++1TnybRWMiv1YN2UW5hyicLSm3AhDaU/89lFkZC//8=,41,30,149,0,1,2015-12-13,2015-12-13,1
281,++2axpngZEynlxNr1+AkwgHHfaEZ/EeOj6Q284RiAkw=,41,30,149,149,1,2016-02-25,2016-02-25,1
285,++2jS71kxhJr7dAl9q0vd6SgNfUY8HfASsF6ItNTYCo=,41,30,149,149,1,2015-02-17,2015-02-16,1
287,++2m3qvMRunJP9QTvQC5ex57i0qYG2HoTj6ZC8it+R0=,41,30,149,0,1,2016-01-10,2016-01-10,1


In [70]:
df = transactions[transactions.msno == '++4cUL0b9CfW8cj0A/wfSxQc4k4fcVtWcLqk2UOdpKs=']

In [71]:
pd.crosstab(df.transaction_date,df.is_cancel)

is_cancel,0,1
transaction_date,,
2015-01-08,1,0
2015-02-08,1,0
2015-03-08,1,0
2015-04-08,1,0
2015-05-08,1,0
2015-06-08,1,0
2015-07-08,1,0
2015-08-08,1,0
2015-09-08,1,0


In [72]:
cnt

,msno,is_auto_renew,cnt
0,++4cUL0b9CfW8cj0A/wfSxQc4k4fcVtWcLqk2UOdpKs=,0,1
1,++4cUL0b9CfW8cj0A/wfSxQc4k4fcVtWcLqk2UOdpKs=,1,28


In [95]:
df = transactions[transactions.msno == '++4cUL0b9CfW8cj0A/wfSxQc4k4fcVtWcLqk2UOdpKs=']
def make_order_number(x):
    x['order_number'] = [i+1 for i in range(x.shape[0])]
    return x
df = df.groupby('msno').apply(make_order_number)
input_col = ['msno','transaction_date','is_cancel','is_auto_renew','order_number']

df[input_col]

,msno,transaction_date,is_cancel,is_auto_renew,order_number
448,++4cUL0b9CfW8cj0A/wfSxQc4k4fcVtWcLqk2UOdpKs=,2015-01-08,0,1,1
449,++4cUL0b9CfW8cj0A/wfSxQc4k4fcVtWcLqk2UOdpKs=,2015-02-08,0,1,2
450,++4cUL0b9CfW8cj0A/wfSxQc4k4fcVtWcLqk2UOdpKs=,2015-03-08,0,1,3
451,++4cUL0b9CfW8cj0A/wfSxQc4k4fcVtWcLqk2UOdpKs=,2015-04-08,0,1,4
452,++4cUL0b9CfW8cj0A/wfSxQc4k4fcVtWcLqk2UOdpKs=,2015-05-08,0,1,5
453,++4cUL0b9CfW8cj0A/wfSxQc4k4fcVtWcLqk2UOdpKs=,2015-06-08,0,1,6
454,++4cUL0b9CfW8cj0A/wfSxQc4k4fcVtWcLqk2UOdpKs=,2015-07-08,0,1,7
455,++4cUL0b9CfW8cj0A/wfSxQc4k4fcVtWcLqk2UOdpKs=,2015-08-08,0,1,8
456,++4cUL0b9CfW8cj0A/wfSxQc4k4fcVtWcLqk2UOdpKs=,2015-09-08,0,1,9
457,++4cUL0b9CfW8cj0A/wfSxQc4k4fcVtWcLqk2UOdpKs=,2015-10-08,0,1,10


In [96]:
df = df[input_col]

In [176]:
df[df.msno == '++5wYjoMgQHoRuD3GbbvmphZbBBwymzv5Q4l8sywtuU=']

,msno,transaction_date,is_cancel,is_auto_renew,order_number
747190,++5wYjoMgQHoRuD3GbbvmphZbBBwymzv5Q4l8sywtuU=,2015-01-31,0.0,1.0,1
747191,++5wYjoMgQHoRuD3GbbvmphZbBBwymzv5Q4l8sywtuU=,2015-02-28,0.0,1.0,2
747192,++5wYjoMgQHoRuD3GbbvmphZbBBwymzv5Q4l8sywtuU=,2015-03-31,0.0,1.0,3
747193,++5wYjoMgQHoRuD3GbbvmphZbBBwymzv5Q4l8sywtuU=,2015-04-30,0.0,1.0,4
747194,++5wYjoMgQHoRuD3GbbvmphZbBBwymzv5Q4l8sywtuU=,2015-05-12,1.0,1.0,5
747195,++5wYjoMgQHoRuD3GbbvmphZbBBwymzv5Q4l8sywtuU=,2015-06-22,0.0,1.0,6
747196,++5wYjoMgQHoRuD3GbbvmphZbBBwymzv5Q4l8sywtuU=,2015-06-30,0.0,1.0,7
747197,++5wYjoMgQHoRuD3GbbvmphZbBBwymzv5Q4l8sywtuU=,2015-07-31,0.0,1.0,8
747198,++5wYjoMgQHoRuD3GbbvmphZbBBwymzv5Q4l8sywtuU=,2015-08-31,0.0,1.0,9
747199,++5wYjoMgQHoRuD3GbbvmphZbBBwymzv5Q4l8sywtuU=,2015-09-30,0.0,1.0,10


# is_cancel

In [190]:
cnt = df.groupby(['msno', 'is_cancel']).size()
cnt.name = 'cnt'
cnt = cnt.reset_index()
cnt
# chance
user_onb_max = df.groupby('msno').order_number.max().reset_index()
user_onb_max.columns = ['msno', 'onb_max']
user_onb_max
user_is_cancel_min = df.groupby(['msno', 'is_cancel']).order_number.min().reset_index()
user_is_cancel_min.columns = ['msno', 'is_cancel', 'onb_min']

chance = pd.merge(user_is_cancel_min, user_onb_max, on='msno', how='left')
chance['is_cancel_chance'] = chance.onb_max - chance.onb_min +1

tbl = pd.merge(cnt, chance, on= ['msno', 'is_cancel'], how='left')
tbl['is_cancel_ratio_by_chance'] = tbl.cnt / tbl.is_cancel_chance
col = ['msno', 'is_cancel_chance', 'is_cancel_ratio_by_chance']
#tbl = tbl[col]
#tbl = tbl.rename(columns = {'chance':'order_number'})[col]

In [191]:
tbl[tbl.msno == '++5wYjoMgQHoRuD3GbbvmphZbBBwymzv5Q4l8sywtuU=']

,msno,is_cancel,cnt,onb_min,onb_max,is_cancel_chance,is_cancel_ratio_by_chance
26,++5wYjoMgQHoRuD3GbbvmphZbBBwymzv5Q4l8sywtuU=,0.0,20,1,21,21,0.952381
27,++5wYjoMgQHoRuD3GbbvmphZbBBwymzv5Q4l8sywtuU=,1.0,1,5,21,17,0.058824


In [193]:
df[df.msno == '++5wYjoMgQHoRuD3GbbvmphZbBBwymzv5Q4l8sywtuU=']

,msno,transaction_date,is_cancel,is_auto_renew,order_number
747190,++5wYjoMgQHoRuD3GbbvmphZbBBwymzv5Q4l8sywtuU=,2015-01-31,0.0,1.0,1
747191,++5wYjoMgQHoRuD3GbbvmphZbBBwymzv5Q4l8sywtuU=,2015-02-28,0.0,1.0,2
747192,++5wYjoMgQHoRuD3GbbvmphZbBBwymzv5Q4l8sywtuU=,2015-03-31,0.0,1.0,3
747193,++5wYjoMgQHoRuD3GbbvmphZbBBwymzv5Q4l8sywtuU=,2015-04-30,0.0,1.0,4
747194,++5wYjoMgQHoRuD3GbbvmphZbBBwymzv5Q4l8sywtuU=,2015-05-12,1.0,1.0,5
747195,++5wYjoMgQHoRuD3GbbvmphZbBBwymzv5Q4l8sywtuU=,2015-06-22,0.0,1.0,6
747196,++5wYjoMgQHoRuD3GbbvmphZbBBwymzv5Q4l8sywtuU=,2015-06-30,0.0,1.0,7
747197,++5wYjoMgQHoRuD3GbbvmphZbBBwymzv5Q4l8sywtuU=,2015-07-31,0.0,1.0,8
747198,++5wYjoMgQHoRuD3GbbvmphZbBBwymzv5Q4l8sywtuU=,2015-08-31,0.0,1.0,9
747199,++5wYjoMgQHoRuD3GbbvmphZbBBwymzv5Q4l8sywtuU=,2015-09-30,0.0,1.0,10


In [120]:
tbl = pd.merge(df,tbl, on = ['msno','is_cancel'])
tbl.tail(n= 1)

,msno,transaction_date,is_cancel,is_auto_renew,order_number,is_cancel_ratio_bychance
1,++4cUL0b9CfW8cj0A/wfSxQc4k4fcVtWcLqk2UOdpKs=,2017-03-26,0,1,29,0.931034


# is_auto_renewal

In [79]:
df = df.groupby('msno').apply(make_order_number)
df

,msno,is_cancel,cnt,onb_min,onb_max,chance,is_cancel_ratio_bychance,order_number
0,++4cUL0b9CfW8cj0A/wfSxQc4k4fcVtWcLqk2UOdpKs=,0,27,1,29,29,0.931034,1
1,++4cUL0b9CfW8cj0A/wfSxQc4k4fcVtWcLqk2UOdpKs=,1,2,15,29,15,0.133333,2


In [78]:
cnt = df.groupby(['msno', 'is_auto_renew']).size()
cnt.name = 'cnt'
cnt = cnt.reset_index()

# chance
user_onb_max = df.groupby('msno').order_number.max().reset_index()
user_onb_max.columns = ['msno', 'onb_max']
user_onb_max
user_is_cancel_min = df.groupby(['msno', 'is_auto_renew']).order_number.min().reset_index()
user_is_cancel_min.columns = ['msno', 'is_auto_renew', 'onb_min']

chance = pd.merge(user_is_cancel_min, user_onb_max, on='msno', how='left')
chance['chance'] = chance.onb_max - chance.onb_min +1

tbl = pd.merge(cnt, chance, on=['msno', 'is_auto_renew'], how='left')
tbl['is_auto_renew_ratio_bychance'] = tbl.cnt / tbl.chance
tbl

AttributeError: 'DataFrameGroupBy' object has no attribute 'order_number'

In [278]:
T = 0
folder = 'trainW-'+str(T)
train = pd.read_csv('../../input/preprocessed_data/trainW-{0}.csv'.format(T))[['msno','is_churn']]
##################################################
# Load  
##################################################
input_col = ['msno','transaction_date','is_cancel','is_auto_renew']
transactions = utils.read_multiple_csv('../../input/preprocessed_data/transactions') # 20,000,000

transactions = transactions.head(n = 5000)
##################################################
# Convert string to datetime format
##################################################
transactions['transaction_date']  = transactions.transaction_date.apply(lambda x: datetime.strptime(x, '%Y-%m-%d'))

df = pd.merge(train, 
transactions[(transactions.transaction_date < datetime.strptime('2017-03-01', '%Y-%m-%d'))], 
on=['msno'], 
how='left')

100%|██████████| 4/4 [00:26<00:00,  6.70s/it]


In [279]:
df = df.dropna()

# all history

In [280]:
df = df.groupby('msno').apply(make_order_number)

In [281]:
tbl_ = df.groupby('msno').is_cancel.sum().to_frame()
tbl_.columns = ['cancel_total_count']
tbl_.reset_index(inplace = True)

In [288]:
df = df.groupby('msno').apply(make_order_number)
#count
cnt = df.groupby(['msno', 'is_cancel']).size()
cnt.name = 'cnt'
cnt = cnt.reset_index()
cnt
# chance
user_onb_max = df.groupby('msno').order_number.max().reset_index()
user_onb_max.columns = ['msno', 'onb_max']
user_onb_max
user_is_cancel_min = df.groupby(['msno', 'is_cancel']).order_number.min().reset_index()
user_is_cancel_min.columns = ['msno', 'is_cancel', 'onb_min']

chance = pd.merge(user_is_cancel_min, user_onb_max, on='msno', how='left')
chance['is_cancel_chance'] = chance.onb_max - chance.onb_min +1

tbl = pd.merge(cnt, chance, on= ['msno', 'is_cancel'], how='left')
tbl['cancel_ratio_by_chance'] = tbl.cnt / tbl.is_cancel_chance
# total_count
tbl_ = df.groupby('msno').is_cancel.sum().to_frame()
tbl_.columns = ['cancel_total_count']
tbl_['cancel_total_count_ratio'] = df.groupby('msno').is_cancel.mean()
tbl_.reset_index(inplace = True)
tbl = pd.merge(tbl, tbl_, on = 'msno')
col = ['msno', 'is_cancel_chance', 'cancel_ratio_by_chance','cancel_total_count','cancel_total_count_ratio']
tbl = tbl[col]
tbl.drop_duplicates('msno',keep = 'last',inplace = True) # 只要is_cancel == 1

In [289]:
tbl
# cancel_ratio

,msno,is_cancel_chance,cancel_ratio_by_chance,cancel_total_count,cancel_total_count_ratio
0,+++hVY1rZox/33YtvDgmKA2Frg/2qhkz12B9ylCvh8o=,4,1.000000,0.0,0.000000
1,+++l/EXNMLTijfLBa8p2TUVVVp2aFGSuUI/h7mLmthw=,20,1.000000,0.0,0.000000
2,+++snpr7pmobhLKUgSHTv/mpkqgBT0tQJ0zQj6qKrqc=,26,1.000000,0.0,0.000000
3,++/9R3sX37CjxbY/AaGvbwr3QkwElKBCtSvVzhCBDOk=,12,1.000000,0.0,0.000000
4,++/UDNo9DLrxT8QVGiDi1OnWfczAdEwThaVyD0fXO50=,20,1.000000,0.0,0.000000
5,++/ZHqwUNa7U21Qz+zqteiXlZapxey86l6eEorrak/g=,22,1.000000,0.0,0.000000
6,++0/NopttBsaAn6qHZA2AWWrDg7Me7UOMs1vsyo4tSI=,12,1.000000,0.0,0.000000
7,++0BJXY8tpirgIhJR14LDM1pnaRosjD1mdO1mIKxlJA=,16,1.000000,0.0,0.000000
8,++0EzISdtKY48Z0GY62jer/LFQwrNIAbADdtU5xStGY=,6,1.000000,0.0,0.000000
9,++0wqjjQge1mBBe5r4ciHGKwtF/m322zkra7CK8I+Mw=,16,1.000000,0.0,0.000000


In [287]:
df[df.msno == '+++hVY1rZox/33YtvDgmKA2Frg/2qhkz12B9ylCvh8o=']

,msno,is_churn,payment_method_id,payment_plan_days,plan_list_price,actual_amount_paid,is_auto_renew,transaction_date,membership_expire_date,is_cancel,order_number
686849,+++hVY1rZox/33YtvDgmKA2Frg/2qhkz12B9ylCvh8o=,0,41.0,30.0,99.0,99.0,1.0,2016-11-16,2016-12-15,0.0,1
686850,+++hVY1rZox/33YtvDgmKA2Frg/2qhkz12B9ylCvh8o=,0,41.0,30.0,99.0,99.0,1.0,2016-12-15,2017-01-15,0.0,2
686851,+++hVY1rZox/33YtvDgmKA2Frg/2qhkz12B9ylCvh8o=,0,41.0,30.0,99.0,99.0,1.0,2017-01-15,2017-02-15,0.0,3
686852,+++hVY1rZox/33YtvDgmKA2Frg/2qhkz12B9ylCvh8o=,0,41.0,30.0,99.0,99.0,1.0,2017-02-15,2017-03-15,0.0,4


In [285]:
tbl[tbl.msno == '++mnT+fA+ilPMzHgEhsyS91vWjM7iRy7//aq/eVTm6s=']

,msno,is_cancel_chance,cancel_ratio_by_chance,cancel_total_count
198,++mnT+fA+ilPMzHgEhsyS91vWjM7iRy7//aq/eVTm6s=,14,0.142857,2.0


In [286]:
tbl[tbl.msno.duplicated()]

,msno,is_cancel_chance,cancel_ratio_by_chance,cancel_total_count


In [131]:
def near(x, keep = 5):
    return x.tail(keep)
#df_ = df.groupby('msno').apply(near,5).reset_index(drop = True)

# near5 

In [207]:
df_ = df.groupby('msno').apply(near,5).reset_index(drop = True)
df_ = df_.groupby('msno').apply(make_order_number)
#count
cnt = df_.groupby(['msno', 'is_cancel']).size()
cnt.name = 'cnt'
cnt = cnt.reset_index()
cnt
# chance
user_onb_max = df_.groupby('msno').order_number.max().reset_index()
user_onb_max.columns = ['msno', 'onb_max']
user_onb_max
user_is_cancel_min = df_.groupby(['msno', 'is_cancel']).order_number.min().reset_index()
user_is_cancel_min.columns = ['msno', 'is_cancel', 'onb_min']

chance = pd.merge(user_is_cancel_min, user_onb_max, on='msno', how='left')
chance['is_cancel_chance'] = chance.onb_max - chance.onb_min +1

tbl = pd.merge(cnt, chance, on= ['msno', 'is_cancel'], how='left')
tbl['cancel_ratio_by_chance'] = tbl.cnt / tbl.is_cancel_chance
col = ['msno', 'is_cancel_chance', 'cancel_ratio_by_chance']
tbl = tbl[col]
tbl.drop_duplicates('msno',keep = 'last',inplace = True) # 只要is_cancel == 1

In [210]:
tbl[tbl.msno == '++mnT+fA+ilPMzHgEhsyS91vWjM7iRy7//aq/eVTm6s=']

,msno,is_cancel_chance,cancel_ratio_by_chance
168,++mnT+fA+ilPMzHgEhsyS91vWjM7iRy7//aq/eVTm6s=,3,0.333333


In [209]:
tbl[tbl.msno.duplicated()]

,msno,is_cancel_chance,cancel_ratio_by_chance


# only 1

In [213]:
df_ = df.groupby('msno').apply(near,1).reset_index(drop = True)
df_ = df_.groupby('msno').apply(make_order_number)
#count
cnt = df_.groupby(['msno', 'is_cancel']).size()
cnt.name = 'cnt'
cnt = cnt.reset_index()
cnt
# chance
user_onb_max = df_.groupby('msno').order_number.max().reset_index()
user_onb_max.columns = ['msno', 'onb_max']
user_onb_max
user_is_cancel_min = df_.groupby(['msno', 'is_cancel']).order_number.min().reset_index()
user_is_cancel_min.columns = ['msno', 'is_cancel', 'onb_min']

chance = pd.merge(user_is_cancel_min, user_onb_max, on='msno', how='left')
chance['is_cancel_chance'] = chance.onb_max - chance.onb_min +1

tbl = pd.merge(cnt, chance, on= ['msno', 'is_cancel'], how='left')
tbl['cancel_ratio_by_chance'] = tbl.cnt / tbl.is_cancel_chance
col = ['msno', 'is_cancel_chance', 'cancel_ratio_by_chance']
tbl = tbl[col]
tbl.drop_duplicates('msno',keep = 'last',inplace = True) # 只要is_cancel == 1

In [214]:
tbl[tbl.msno == '++mnT+fA+ilPMzHgEhsyS91vWjM7iRy7//aq/eVTm6s=']

,msno,is_cancel_chance,cancel_ratio_by_chance
162,++mnT+fA+ilPMzHgEhsyS91vWjM7iRy7//aq/eVTm6s=,1,1.0


In [215]:
tbl[tbl.msno.duplicated()]

,msno,is_cancel_chance,cancel_ratio_by_chance


In [172]:
##################################################
# only one prvious order
##################################################
df_ = df.groupby('msno').apply(near,1).reset_index(drop = True)
#cnt
cnt = df_.groupby(['msno', 'is_cancel']).size()
cnt.name = 'cnt'
cnt = cnt.reset_index()
# chance
user_onb_max = df_.groupby('msno').order_number.max().reset_index()
user_onb_max.columns = ['msno', 'onb_max']
user_onb_max
user_is_cancel_min = df_.groupby(['msno', 'is_cancel']).order_number.min().reset_index()
user_is_cancel_min.columns = ['msno', 'is_cancel', 'onb_min']

chance = pd.merge(user_is_cancel_min, user_onb_max, on='msno', how='left')
chance['chance'] = chance.onb_max - chance.onb_min +1

tbl = pd.merge(cnt, chance, on= ['msno', 'is_cancel'], how='left')
tbl['is_cancel_ratio_by_chance'] = tbl.cnt / tbl.chance
col = ['msno', 'order_number', 'is_cancel_ratio_by_chance']
tbl = tbl.rename(columns = {'chance':'order_number'})[col]
tbl = pd.merge(df_,tbl, on = ['msno'])
tbl = tbl[output]
tbl.drop_duplicates('msno',keep = 'last', inplace = True)
# print ('fucking', tbl)
tbl

,msno,transaction_date,is_cancel,is_auto_renew,order_number_x,order_number_y,is_cancel_ratio_by_chance
0,+++hVY1rZox/33YtvDgmKA2Frg/2qhkz12B9ylCvh8o=,2017-02-15,0.0,1.0,4,1,1.0
1,+++l/EXNMLTijfLBa8p2TUVVVp2aFGSuUI/h7mLmthw=,2017-02-28,0.0,1.0,20,1,1.0
2,+++snpr7pmobhLKUgSHTv/mpkqgBT0tQJ0zQj6qKrqc=,2017-02-26,0.0,1.0,26,1,1.0
3,++/9R3sX37CjxbY/AaGvbwr3QkwElKBCtSvVzhCBDOk=,2017-02-15,0.0,1.0,12,1,1.0
4,++/UDNo9DLrxT8QVGiDi1OnWfczAdEwThaVyD0fXO50=,2017-02-28,0.0,1.0,20,1,1.0
5,++/ZHqwUNa7U21Qz+zqteiXlZapxey86l6eEorrak/g=,2017-02-28,0.0,1.0,22,1,1.0
6,++0/NopttBsaAn6qHZA2AWWrDg7Me7UOMs1vsyo4tSI=,2017-02-20,0.0,1.0,12,1,1.0
7,++0BJXY8tpirgIhJR14LDM1pnaRosjD1mdO1mIKxlJA=,2017-02-24,0.0,0.0,16,1,1.0
8,++0EzISdtKY48Z0GY62jer/LFQwrNIAbADdtU5xStGY=,2016-10-24,0.0,0.0,6,1,1.0
9,++0wqjjQge1mBBe5r4ciHGKwtF/m322zkra7CK8I+Mw=,2017-02-06,0.0,1.0,16,1,1.0


In [171]:
df_[df_.msno == '++MRn/eFl4FPB8NdhvT9LyZXvtF8K48qS8lizaT2WDM=']

,msno,transaction_date,is_cancel,is_auto_renew,order_number
72,++MRn/eFl4FPB8NdhvT9LyZXvtF8K48qS8lizaT2WDM=,2017-02-07,0.0,1.0,13
